In [ ]:
#This notebook contains an implementation to solve the problem min Sum ||A-B_i||_1 + ||2A-11^t||_nucl.
#We solve the problem an the implementation based on "The Augmented Lagrange Multiplier
#method for exact recovery of corrupted low-rank matrices" and some notes of Boyd for ADMM.
#which we link here. https://web.stanford.edu/~boyd/papers/pdf/admm_slides.pdf

#"/although interior point methods normally take very few iterations t converge, they have difficulty in
#handling large matrices because of the complexity of computing the step direction is O(m^6), where m is the dimension of 
#matrix [...] generic interior point solvers are too limited for Robust PCA ".

#Note that the robust PCA is used in many contexts. Ours is the clique problem which deals with nxn symetric matrices
#. However this implementation works for m x n matrices. Our implementation is an extension of the robust pca algorithm.


In [1]:
#admmRecovery does not call mosek. It uses the prox operation to find xi+1 and Z.
function admmRecovery(observations,lambda,n,rho=1.6,stopCrit1=1.0e-5,maxIter=200)
    
    #Important! all matrices in the observations are assumed to have 1 in the diagonal.
    #Transformation of the B_i to the C_i where C_i = 2B_i-11^t
    observationsTransformed = transform(observations,n)
    #Initialization (All parameters are based on the paper.)
    k = 0
    N = length(observationsTransformed)
    averObservations=AvergeMatrix(observationsTransformed)
    Y = Array{Any}(N)
    [Y[r]=observationsTransformed[r]/J(observationsTransformed[r],lambda) for r in 1:N]
    Z= averObservations
    softZ = zeros(Z)
    Zviejo= zeros(Z)
    Xviejo = Array{Any}(N)
    [Xviejo[r]=zeros(Z) for r in 1:N]
    
    Xnuevo = Array{Any}(N)
    [Xnuevo[r]=zeros(Z) for r in 1:N]
    
    #mu = 1.25/norm(averObservations,2)
    #Iterations
    #In order to avoid evaluation of stoping criteria in the first loop, we do a while true and then 
    #add breaks if the stoping criteria are met.
    while true 
                
        # First solve x^i_k+1 = arg min 1/2||x^i_k-C_i||+mu/2||x^i_k-Z_k+y^i_k||_2^2
      #  println("solving for x...")
        [Xnuevo[i] = solveForXprox(observationsTransformed[i],Y[i],Z,rho,n) for i in 1:N]
       

       # println("Solved, now solving for Z")
        tic()
         # now we solve Z_k+1 = arg min prox_g,rho (1/N sum(x_i_k+1 + 1/rho y_i^k))
        singValDesc = svdfact!(AvergeMatrix(Xnuevo +(1/rho)*Y))
        #SVD(A) returns a triple (U,S,V) where S contains the singular values and A=U*S*V' (' denotes ')
        #for SVD(A) to work correctly, A must be m x n where m>= n or else S wont have the proper dimensions.
        softZ = diagm(SoftThreshold.(singValDesc[:S],lambda/rho))
        softZ = convert(Array{Float64,2},softZ)    
        Z = singValDesc[:U]*softZ*singValDesc[:Vt]       
        toc()
       # println("solved. now solving Y")
        #update Y
        [Y[i] = Y[i]+ rho*(Xnuevo[i]-Z)for i in 1:N]
        
        
        #compute the residuals to check stopping conditions and compute rho
        residual = AvergeMatrix(Xnuevo)-Z
        residualDual = rho*(Zviejo-Z)
        #check stopping condition
     #   println("Cheking stop criteriums and updating.")
        if  firstCriterium(stopCrit1,n,residual,residualDual)
     #       println("Done")
           break
        end
        #update rho
        rho = updaterho(rho,residual,residualDual)
         
        #update Xviejo
        Xviejo = Xnuevo
        #update Zviejo
        Zviejo = Z
        # update k 
    #    println("Step $(k)")
    #    println(norm(Xnuevo[1],2))
        k = k+1
        
        println(rho)
        if k>maxIter
            println("exceeded max number of iteration at solving")
            break
        end    
    end
    return((Z,Xnuevo))
end
        
        
        

admmRecovery (generic function with 4 methods)

In [2]:
function solveForXprox(obs,Y,Z,mu,n)
    actualizado = SoftThreshold(Z-(1/mu)*Y-obs,1/(2*mu))+obs
    return(actualizado)
end    


solveForXprox (generic function with 1 method)

In [3]:
#Aux functions


#perturbation operator. Perturbs every entry of the matrix W using the function f.
#W is a m x n matriz to perturb
#epsilon is the perturbation
function SoftThreshold(W,perturbation)
    map(W) do x
        perturb(x,perturbation)
    end
end



#Method for perturbing x given epsilon. Used in the softThresholding function.
function perturb(x,epsilon)
    if x>epsilon 
        return x-epsilon
    end
    if x<-epsilon
        return x+epsilon
    else
        return(0)
    end
end



#Method to average matrices

function AvergeMatrix(observations)
    size = length(observations)
    return (1/size)*sum(observations)
end

#J operator
#W is an m x n matriz
#lambda : parameter of the algorithm
function J(W,lambda)
    return max(norm(W,2),(1/lambda)*maximum(abs.(W)))
end

#Function  transform the observed matrices to the desired form. All observed matrices
# have 1 on the diagonal

function transformMatrix(mat,n)
    return(2*mat-ones(n)*ones(n)') 
end

function transform(observations,n)
    return(transformMatrix.(observations,n))
end



function firstCriterium(stopCrit1,n,residual,residualDual)
    
    if  sqrt(n)*norm(residual,2)<= stopCrit1 && sqrt(n)*norm(residualDual,2)<= stopCrit1
        return(true)
    end
    return false
end

# Updating functions

function updaterho(rho,residual,residualDual)
    rhoNew = rho
    if norm(residual,2)>=10*norm(residualDual,2)
    rhoNew = 2*rho
    end
    if norm(residualDual,2)>=10*norm(residual,2)
        rhoNew = rho/2
    end
    return(rhoNew)
end
        


updaterho (generic function with 1 method)

#admmRecovery does calls mosek to find xi+1 and Z.


function admmRecovery2(observations,lambda,n,rho=1.6,stopCrit1=1.0e-7,stopCrit2=1.0e-5,maxIter=10000)
    
    #Important! all matrices in the observations are assumed to have 1 in the diagonal.
    #Transformation of the B_i to the C_i where C_i = 2B_i-11^t
    observationsTransformed = transform(observations,n)
    #Initialization (All parameters are based on the paper.)
    k = 0
    N = length(observationsTransformed)
    averObservations=AvergeMatrix(observationsTransformed)
    Y = Array{Any}(N)
    [Y[r]=observationsTransformed[r]/J(observationsTransformed[r],lambda) for r in 1:N]
    Z= averObservations
    Xviejo = Array{Any}(N)
    [Xviejo[r]=zeros(Z) for r in 1:N]
    
    Xnuevo = Array{Any}(N)
    [Xnuevo[r]=zeros(Z) for r in 1:N]
    
    #mu = 1.25/norm(averObservations,2)
    mu=0.1
    #Iterations
    #In order to avoid evaluation of stoping criteria in the first loop, we do a while true and then 
    #add breaks if the stoping criteria are met.
    while true 
        
       
        
        # First solve x^i_k+1 = arg min ||x^i_k-C_i||+ <yi^k,xi-z^k>   +1/2||x^i_k-Z_k||_2^2
        #for now, we call mosek to solve this
            [Xnuevo[i] = solveForXMosek(observationsTransformed[i],Y[i],Z,mu,n) for i in 1:N]

            

         # now we solve Z_k+1 = arg min prox_g,rho (1/N sum(x_i_k+1 + 1/rho y_i^k))
        singValDesc = svd(AvergeMatrix(Xnuevo) +(1/mu)*AvergeMatrix(Y))
        #SVD(A) returns a triple (U,S,V) where S contains the singular values and A=U*S*V' (' denotes ')
        #for SVD(A) to work correctly, A must be m x n where m>= n or else S wont have the proper dimensions.
        Z = singValDesc[1]*SoftThreshold(diagm(singValDesc[2]),lambda/(mu*N))*singValDesc[3]'

        #update Y
        [Y[i] = Y[i]+ mu*(Xnuevo[i]-Z)for i in 1:N]
        
        #check stopping condition
      #  if  firstCriterium(stopCrit1,Z,averObservations,Xnuevo)
       #     println("Done")
        #    break
       # end
        #update mu
        #mu = updateMu(mu,rho,Xviejo,Xnuevo,averObservations,stopCrit2)
        #mu = mu*rho
        #update Xviejo
        Xviejo = Xnuevo
        # update k 
        println("Step $(k)")
        println(norm(Xnuevo[1],2))
        k = k+1
        
        println(mu)
        if k>maxIter
            println("exceeded max number of iteration at solving")
            break
        end    
    end
    return((Z,Xnuevo))
end
        

#observations are the observations to compute the function f_i.
#Y is the dual in step k.
#X are the old values of x.
#Z is the objective variable in step k
#mu is the step size.
#using JuMP 
#using Mosek

function solveForXMosek(obs,Y,Z,mu,n)
    m = Model(solver=MosekSolver())
    @variable(m,x[1:n,1:n])
    @variable(m,s[1:n,1:n])
    @constraint(m,-s.<=x-obs)    
    @constraint(m,x-obs.<=s)
    @objective(m,Min,0.5*trace(s'*(ones(n)*ones(n)'))+trace(Y'*(x-Z))+(mu/2)*(trace((x-Z)'*(x-Z))))
     status = solve(m)
    return(getvalue(x))
end
   



<h1>Test<h1>

In [4]:
#using NBInclude
#nbinclude("AuxFunctions.ipynb")
#nbinclude("GraphLearning.ipynb")

evaluarADMMGraphLearning4 (generic function with 2 methods)

In [5]:
#srand(12536)

MersenneTwister(UInt32[0x000030f8], Base.dSFMT.DSFMT_state(Int32[-642584449, 1073679535, 1179766305, 1073019918, 1125216476, 1073564878, 957469279, 1073717922, 279120579, 1073148466  …  800267814, 1073005159, -793867396, 1073706843, -1618855785, -1651276217, 355150608, -186995734, 382, 0]), [1.06397, 1.96936, 1.23695, 1.49785, 1.18087, 1.86879, 1.88006, 1.56963, 1.25329, 1.73932  …  1.30971, 1.95197, 1.43206, 1.10679, 1.3675, 1.55442, 1.41634, 1.31668, 1.76075, 1.90799], 382)

In [6]:

#grafo20obs5 = Array{Any}(5)
#grafo20obs5[1]=erdosgraph([0.77,0.77],0.1,[12,8],20)+diagm(ones(20))
#grafo20obs5[2]=erdosgraph([0.77,0.77],0.1,[12,8],20)+diagm(ones(20))
#grafo20obs5[3]=erdosgraph([0.77,0.77],0.1,[12,8],20)+diagm(ones(20))
#grafo20obs5[4]=erdosgraph([0.77,0.77],0.1,[12,8],20)+diagm(ones(20))
#grafo20obs5[5]=erdosgraph([0.77,0.77],0.1,[12,8],20)+diagm(ones(20))
#1

1

In [ ]:
#0.28889 s
#tic()
#recuperaRegular = learnRegularized2A([1/5,1/5,1/5,1/5,1/5],1.6,grafo20obs5,20)
#toc()

In [ ]:
#recuperaRegular[1]

In [ ]:
#function admmRecovery2(observations,lambda,n,rho=1.6,stopCrit1=1.0e-7,stopCrit2=1.0e-5,maxIter=10000)


In [9]:
#64.3 segundos
#tic()
#respuestaAdmm = admmRecovery2(grafo20obs5,1.6,20,1.6,1.0e-5,1.0e-5,500)
#toc()

In [31]:
#bli =(respuestaAdmm[1]+ones(20)*ones(20)')/2

20×20 Array{Float64,2}:
 1.0          1.0          1.0         …   3.83932e-9    4.05544e-9 
 1.0          1.0          1.0            -1.2696e-9    -1.05348e-9 
 1.0          1.0          1.0             3.81038e-9    4.0265e-9  
 1.0          1.0          1.0             4.65253e-9    4.86865e-9 
 1.0          1.0          1.0             2.90721e-9    3.12333e-9 
 1.0          1.0          1.0         …   9.15788e-9    9.374e-9   
 1.0          1.0          1.0            -7.20876e-10  -5.04757e-10
 1.0          1.0          1.0            -6.42067e-10  -4.25948e-10
 1.0          1.0          1.0            -1.22293e-9   -1.00681e-9 
 1.0          1.0          1.0            -6.05223e-10  -3.89105e-10
 1.0          1.0          1.0         …  -3.21656e-10  -1.05537e-10
 1.0          1.0          1.0            -1.09189e-9   -8.75772e-10
 3.99024e-9  -1.11868e-9   3.9613e-9       1.0           1.0        
 3.95991e-9  -1.14902e-9   3.93096e-9      1.0           1.0        
 3.86594e-

In [ ]:
#0.069 s
#tic()
#respuestaAdmmProx = admmRecovery(grafo20obs5,1.6,20,1.6,1.0e-5,500)
#toc()

In [13]:
#bli =(respuestaAdmmProx[1]+ones(20)*ones(20)')/2

20×20 Array{Float64,2}:
 1.0         1.0          1.0         …   2.85835e-9    1.41128e-8
 1.0         1.0          1.0             2.58315e-8    3.70859e-8
 1.0         1.0          1.0            -6.70257e-9    4.55191e-9
 1.0         1.0          1.0            -6.56059e-9    4.6939e-9 
 1.0         1.0          1.0            -3.61625e-9    7.63823e-9
 1.0         1.0          1.0         …  -2.22011e-8   -1.09466e-8
 1.0         1.0          1.0             6.73688e-9    1.79914e-8
 1.0         1.0          1.0             1.25013e-8    2.37558e-8
 1.0         1.0          1.0            -5.64038e-9    5.6141e-9 
 1.0         1.0          1.0             1.45814e-9    1.27126e-8
 1.0         1.0          1.0         …   8.55231e-11   1.134e-8  
 1.0         1.0          1.0            -5.22803e-9    6.02645e-9
 8.07269e-9  3.10458e-8  -1.48824e-9      1.0           1.0       
 1.36442e-8  3.66173e-8   4.08325e-9      1.0           1.0       
 3.07075e-8  5.36806e-8   2.11466e-8  

<h2>Grafos grandes<h2>

In [14]:
#grafo200obs3 = Array{Any}(3)
#grafo200obs3[1]=erdosgraph([0.85,0.85],0.1,[100,100],200)+diagm(ones(200))
#grafo200obs3[2]=erdosgraph([0.85,0.85],0.1,[100,100],200)+diagm(ones(200))
#grafo200obs3[3]=erdosgraph([0.85,0.85],0.1,[100,100],200)+diagm(ones(200))


200×200 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  1.0  1.0  1.0  1.0  0.0     0.0  0.0  0.0  1.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  1.0  0.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  0.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.

In [ ]:
#7 horas
#tic()
#respuestaAdmmGrande = admmRecovery2(grafo200obs3,1.6,200,1.6,1.0e-5,1.0e-5,200)
#toc()

In [12]:
#(respuestaAdmmGrande[1]+ones(200)*ones(200)')/2

200×200 Array{Float64,2}:
  0.999991     0.999994     1.00001     …   2.22356e-5   3.47044e-6
  0.999994     0.999998     1.0             1.56104e-5   1.08383e-6
  1.00001      1.0          0.99999        -2.43845e-5  -2.54589e-6
  1.0          1.0          0.999999       -2.01411e-6  -3.71595e-7
  1.00001      6.91444e-6   0.999984       -3.77345e-5  -3.92154e-6
  0.999996     0.999998     1.0         …   7.58836e-6   8.0147e-7 
  1.00001      1.0          0.999983       -2.61186e-5  -2.72654e-6
  1.00001      1.0          0.99999        -2.41558e-5  -3.70371e-6
  1.00001      1.0          0.999994       -1.41018e-5  -1.49836e-6
  0.999987     0.999994     2.19287e-5      3.39515e-5   3.59029e-6
  1.00001      1.0          0.999993    …  -1.7197e-5   -1.77179e-6
  1.00001      1.0          0.99999        -2.24458e-5  -3.4477e-6 
  0.999973    -1.25637e-5   2.95379e-5      4.57386e-5   7.21631e-6
  ⋮                                     ⋱                          
  9.35394e-6   6.57673

In [ ]:
#3.15 s (antes de la correccion del cast tardabamos 7 minutos)
# sin embargo notamos que desde el step 20 ya no cambia la norma de la matriz asi que volvemos a tratar mas abajo...
#tic()
#respuestaAdmmProx = admmRecovery(grafo200obs3,1.6,200,1.6,1.0e-5,200)
#toc()

In [18]:
#(respuestaAdmmProx[1]+ones(200)*ones(200)')/2

200×200 Array{Float64,2}:
  1.0           1.0           1.0          …  -8.55664e-10  -4.35249e-10
  1.0           1.0           1.0             -1.42543e-9   -1.00501e-9 
  1.0           1.0           1.0             -1.25792e-9   -8.37503e-10
  1.0           1.0           1.0             -5.94895e-10  -1.74481e-10
  1.0           1.0           1.0             -8.41516e-10  -4.21101e-10
  1.0           1.0           1.0          …  -1.3352e-9    -9.14786e-10
  1.0           1.0           1.0             -1.33125e-9   -9.10839e-10
  1.0           1.0           1.0             -1.21179e-9   -7.91379e-10
  1.0           1.0           1.0             -1.19742e-9   -7.77007e-10
  1.0           1.0           1.0             -8.47194e-10  -4.2678e-10 
  1.0           1.0           1.0          …  -1.15166e-9   -7.31249e-10
  1.0           1.0           1.0             -2.35398e-10   1.85017e-10
  1.0           1.0           1.0             -1.34595e-9   -9.25532e-10
  ⋮                      

<h3>n=1000<h3>

In [19]:
#grafo1000obs3 = Array{Any}(3)
#grafo1000obs3[1]=erdosgraph([0.85,0.85],0.1,[250,750],1000)+diagm(ones(1000))
#grafo1000obs3[2]=erdosgraph([0.85,0.85],0.1,[250,750],1000)+diagm(ones(1000))
#grafo1000obs3[3]=erdosgraph([0.85,0.85],0.1,[250,750],1000)+diagm(ones(1000))


1000×1000 Array{Float64,2}:
 1.0  1.0  1.0  0.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  1.0  0.0  0.0  1.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  1.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  1.0  1.0  0.0  

In [ ]:
#139 min = 2h
#tic()
#respuestaAdmmProx = admmRecovery(grafo1000obs3,1.6,1000,1.6,1.0e-5,1.0e-5,30)
#toc()

In [13]:
#bli =(respuestaAdmmProx[1]+ones(1000)*ones(1000)')/2

1000×1000 Array{Float64,2}:
  0.999698      0.999597      1.00019      …   5.27828e-5    9.69153e-5 
  0.999597      0.999937      1.00039         -0.000339631   0.165447   
  1.00019       1.00039       0.999527        -0.000176023   1.17976e-5 
  1.0011        0.99982       0.999582         0.000361548   0.000251315
  0.999945      0.999489      1.00069         -2.37356e-5   -0.000138101
  0.999375      1.00016       1.00028      …   6.34709e-5   -8.31634e-5 
  0.99994       1.00036       1.00013          0.000152232  -0.000208594
  1.00002       1.00003       1.00013         -0.000155825   0.000178341
  0.999747      1.00024       0.999945        -0.000148351  -0.000212509
  0.999784      0.999806      0.999676        -0.000156711   4.00402e-5 
  0.999924      1.00026       0.999705     …   7.88072e-5    5.69703e-5 
  0.999948      0.99988       1.00087          1.29156e-5   -0.000121226
  0.999494      1.00011       0.763906        -8.92286e-5    8.18885e-6 
  ⋮                    

In [ ]:
#corrigiendo con el cast
#9.26 min
#tic()
#respuestaAdmmProx = admmRecovery(grafo1000obs3,2,1000,1.6,1.0e-5,200)
#toc()


In [23]:
#bli =(respuestaAdmmProx[1]+ones(1000)*ones(1000)')/2

1000×1000 Array{Float64,2}:
  1.0          1.0           1.0          …  -1.86517e-14   1.53788e-12
  1.0          1.0           1.0              1.12854e-11   1.28419e-11
  1.0          1.0           1.0             -1.77698e-11  -1.62133e-11
  1.0          1.0           1.0             -1.63668e-11  -1.48103e-11
  1.0          1.0           1.0             -1.80876e-11  -1.65311e-11
  1.0          1.0           1.0          …   2.53103e-12   4.08756e-12
  1.0          1.0           1.0             -1.71191e-11  -1.55626e-11
  1.0          1.0           1.0             -1.82464e-11  -1.66899e-11
  1.0          1.0           1.0             -1.71922e-11  -1.56357e-11
  1.0          1.0           1.0             -1.64335e-12  -8.68194e-14
  1.0          1.0           1.0          …  -1.99573e-11  -1.84007e-11
  1.0          1.0           1.0             -1.85243e-11  -1.69678e-11
  1.0          1.0           1.0             -1.88827e-11  -1.73261e-11
  ⋮                                 